In [1]:
import pandas as pd
import numpy as np
sf3 = pd.read_csv('data/SHARADAR_holdings.csv')
sf3 = sf3.dropna()

In [2]:
df=sf3

df[['ticker', 'investorname', 'calendardate']] = df[['ticker', 'investorname', 'calendardate']].astype(str)
df[['value', 'units', 'price']] = df[['value', 'units', 'price']].astype(float)
df=df[df['securitytype']=='SHR']
df=df[['ticker', 'investorname', 'calendardate', 'value', 'units', 'price']]

In [3]:
df_pivot = df.pivot_table(index=['ticker', 'investorname', 'calendardate'])

In [4]:
df_pivot

price    units   value
ticker     investorname             calendardate                        
0001388994 HUNTINGTON NATIONAL BANK 2018-09-30      0.0      1.0     0.0
                                    2018-12-31      0.0      1.0     0.0
                                    2019-06-30      0.0      1.0     0.0
                                    2019-09-30      0.0      1.0     0.0
                                    2019-12-31      0.0      1.0     0.0
...                                                 ...      ...     ...
ZZZOF      IFP ADVISORS INC         2021-09-30      7.0    130.0  1000.0
                                    2021-12-31      7.0    130.0  1000.0
           INVERNESS COUNSEL LLC    2018-03-31      0.0  23800.0  7000.0
                                    2018-06-30      0.0  23800.0  4000.0
                                    2018-09-30      0.0  23800.0  2000.0

[33563204 rows x 3 columns]

In [5]:
# Get unique counts of 'ticker', 'investorname', 'calendardate'
num_tickers = df['ticker'].nunique()
num_investors = df['investorname'].nunique()
num_dates = df['calendardate'].nunique()

# Reshape the values of the DataFrame to a 3D numpy array
#array_3d = df_pivot.values.reshape(num_tickers, num_investors, num_dates, 3)
num_tickers*num_investors*num_dates
df_pivot.shape
#df_pivot.values.reshape(num_tickers, num_investors, num_dates)
#num_tickers*num_investors*num_dates

(33563204, 3)

In [5]:

# Get the number of unique tickers, investors, and dates
num_tickers = df['ticker'].nunique()
num_investors = df['investorname'].nunique()
num_dates = df['calendardate'].nunique()

# Create a 3D numpy array of the same shape
array_3d = np.full((num_tickers, num_investors, num_dates), 0.0,dtype=np.float16)

# Create mappings from tickers, investors, and dates to indices
ticker_to_index = {ticker: i for i, ticker in enumerate(df['ticker'].unique())}
investor_to_index = {investor: i for i, investor in enumerate(df['investorname'].unique())}
date_to_index = {date: i for i, date in enumerate(df['calendardate'].unique())}

# Iterate over the DataFrame rows to fill in the values
count=0
max_count=15
for idx, row in df.iterrows():
    array_3d[ticker_to_index[row['ticker']], investor_to_index[row['investorname']], date_to_index[row['calendardate']]] = row['value']
    print(f'row["value"]={row["value"]}')
    ticker=row["ticker"]
    investorname=row['investorname']
    calenderdate=row["calendardate"]
    array_val=array_3d[ticker_to_index[row['ticker']], investor_to_index[row['investorname']], date_to_index[row['calendardate']]]
    print(f'array_3d[{ticker}][{investorname}][{calenderdate}]={array_val}')
    count+=1
    if (count>max_count):
        break

row["value"]=181944939000.0
array_3d[EAR][PATIENT SQUARE CAPITAL LP][2022-12-31]=inf
row["value"]=166083092402.0
array_3d[AAPL][VANGUARD GROUP INC][2022-12-31]=inf
row["value"]=154123895838.0
array_3d[MSFT][VANGUARD GROUP INC][2022-12-31]=inf
row["value"]=133725037174.0
array_3d[AAPL][BLACKROCK INC][2022-12-31]=inf
row["value"]=127594482097.0
array_3d[MSFT][BLACKROCK INC][2022-12-31]=inf
row["value"]=116305043218.0
array_3d[AAPL][BERKSHIRE HATHAWAY INC][2022-12-31]=inf
row["value"]=80054070850.0
array_3d[GOOGL][VANGUARD GROUP INC][2022-12-31]=inf
row["value"]=76250382632.0
array_3d[AAPL][STATE STREET CORP][2022-12-31]=inf
row["value"]=70644900477.0
array_3d[MSFT][STATE STREET CORP][2022-12-31]=inf
row["value"]=69055938156.0
array_3d[GOOGL][BLACKROCK INC][2022-12-31]=inf
row["value"]=59813885796.0
array_3d[AMZN][VANGUARD GROUP INC][2022-12-31]=inf
row["value"]=54539476000.0
array_3d[ROIV][PATIENT SQUARE CAPITAL LP][2022-12-31]=inf
row["value"]=49956521665.0
array_3d[AMZN][BLACKROCK INC]

/var/folders/c1/dv7w7b2x49j3spqsxnz6y_sw0000gn/T/ipykernel_2792/3472688031.py:18: RuntimeWarning: overflow encountered in cast
  array_3d[ticker_to_index[row['ticker']], investor_to_index[row['investorname']], date_to_index[row['calendardate']]] = row['value']


In [4]:
array_3d.shape
sum(sum(array_3d>0))
array_3d.shape


(15662, 9472, 32)

In [15]:
data_3d={}
data_3d["values_file"]="data/values.npz"
data_3d["values"]=array_3d
data_3d["tickers"]=list(ticker_to_index.keys())
data_3d["investors"]=list(investor_to_index.keys())
data_3d["dates"]=list(date_to_index.keys())
data_3d["ticker_to_index"]=ticker_to_index
data_3d["investor_to_index"]=investor_to_index
data_3d["date_to_index"]=date_to_index


In [17]:
import pickle


# save the dictionary
with open('data/3d_data_with_dimensions.pickle', 'wb') as handle:
    pickle.dump(data_3d, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('data/3d_data_with_dimensions_description.pickle', 'wb') as handle:
#    pickle.dump(data_3d, handle, protocol=pickle.HIGHEST_PROTOCOL)
#np.savez(data_3d["values_file"],array_3d)


In [22]:
with open('data/3d_data_with_dimensions.pickle', 'rb') as f:
    dict_vals= pickle.load(f)
arr_vals=dict_vals["values"]


In [30]:
arr_vals.shape
arr_buy_sell=np.diff(arr_vals,axis=2)


/Users/stirlitz/miniforge3/lib/python3.10/site-packages/numpy/lib/function_base.py:1448: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


In [33]:
array_3d

array([[[  inf,    0.,    0., ...,    0.,    0.,    0.],
        [  inf,   inf,   inf, ...,    0.,    0.,    0.],
        [  inf,   inf,   inf, ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.]],

       [[   0.,    0.,    0., ...,    0.,    0.,    0.],
        [  inf,   inf,   inf, ...,   inf,   inf,   inf],
        [  inf,   inf,   inf, ...,   inf,   inf,   inf],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0., 4624.]],

       [[   0.,    0.,    0., ...,    0.,    0.,    0.],
        [  inf,   inf,   inf, ...,   inf,   inf,   inf],
        [  inf,   inf,   inf, ...,   inf,   inf,   inf],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    

In [16]:
import torch

a = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [10, 8, 9]])


In [21]:
a[0,:]
a.argmax(dim=1)
a.sum(dim=0)
a[0,:]

tensor([1, 2, 3])